In [75]:
import pandas as pd
import numpy as np
import xgboost as xgb


In [76]:
train= pd.read_csv(r'C:\Users\Nishant\Desktop\Junta\Mobility\train.csv')
test= pd.read_csv(r'C:\Users\Nishant\Desktop\Junta\Mobility\test.csv')

In [77]:
train

,Trip_ID,Trip_Distance,Type_of_Cab,Customer_Since_Months,Life_Style_Index,Confidence_Life_Style_Index,Destination_Type,Customer_Rating,Cancellation_Last_1Month,Var1,Var2,Var3,Gender,Surge_Pricing_Type
0,T0005689460,6.77,B,1.0,2.42769,A,A,3.90500,0,40.0,46,60,Female,2
1,T0005689461,29.47,B,10.0,2.78245,B,A,3.45000,0,38.0,56,78,Male,2
2,T0005689464,41.58,NaN,10.0,NaN,NaN,E,3.50125,2,NaN,56,77,Male,2
3,T0005689465,61.56,C,10.0,NaN,NaN,A,3.45375,0,NaN,52,74,Male,3
4,T0005689467,54.95,C,10.0,3.03453,B,A,3.40250,4,51.0,49,102,Male,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131657,T0005908509,11.72,D,1.0,2.74229,A,A,3.28500,0,61.0,47,76,Male,3
131658,T0005908510,74.81,C,7.0,2.81059,C,A,0.44500,0,NaN,63,88,Male,2
131659,T0005908512,40.17,C,10.0,2.99565,B,A,3.33625,0,NaN,48,75,Female,2
131660,T0005908513,46.88,B,4.0,3.04744,A,B,4.15750,1,47.0,54,79,Male,2


In [78]:
data['Var3'].describe()

count    131662.000000
mean         75.099019
std          11.578278
min          52.000000
25%          67.000000
50%          74.000000
75%          82.000000
max         206.000000
Name: Var3, dtype: float64

In [79]:
data.isnull().sum()

Trip_ID                            0
Trip_Distance                      0
Type_of_Cab                    20210
Customer_Since_Months           5920
Life_Style_Index               20193
Confidence_Life_Style_Index    20193
Destination_Type                   0
Customer_Rating                    0
Cancellation_Last_1Month           0
Var1                           71030
Var2                               0
Var3                               0
Gender                             0
Surge_Pricing_Type                 0
dtype: int64

In [80]:
train['Gender'] = train['Gender'].replace(to_replace = {'Male': 0, 'Female': 1}) 
test['Gender'] = test['Gender'].replace(to_replace = {'Male': 0, 'Female': 1})

cab_type = {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4}
train['Type_of_Cab'] = train['Type_of_Cab'].replace(to_replace = cab_type) 
test['Type_of_Cab'] = test['Type_of_Cab'].replace(to_replace = cab_type)

In [84]:
train.isnull().sum()

Trip_ID                            0
Trip_Distance                      0
Type_of_Cab                    20210
Customer_Since_Months           5920
Life_Style_Index               20193
Confidence_Life_Style_Index    20193
Destination_Type                   0
Customer_Rating                    0
Cancellation_Last_1Month           0
Var1                           71030
Var2                               0
Var3                               0
Gender                             0
Surge_Pricing_Type                 0
dtype: int64

In [85]:
confi_index = {'A': 0, 'B': 1, 'C': 2}
train['Confidence_Life_Style_Index'] = train['Confidence_Life_Style_Index'].replace(to_replace = confi_index)
test['Confidence_Life_Style_Index'] = test['Confidence_Life_Style_Index'].replace(to_replace = confi_index)
train['Surge_Pricing_Type'] = train['Surge_Pricing_Type'] - 1

In [86]:
X_train = train.copy()
X_test = test.copy()


In [87]:
from sklearn.preprocessing import LabelEncoder
print("Label Encoding...")
for f in ['Destination_Type']:
    lbl = LabelEncoder()
    lbl.fit(list(X_train[f].values) + list(X_test[f].values)) #transforming values in Destination type using label encoder 
    X_train[f] = lbl.transform(list(X_train[f].values))
    X_test[f] = lbl.transform(list(X_test[f].values))

Label Encoding...


In [88]:
X_train

,Trip_ID,Trip_Distance,Type_of_Cab,Customer_Since_Months,Life_Style_Index,Confidence_Life_Style_Index,Destination_Type,Customer_Rating,Cancellation_Last_1Month,Var1,Var2,Var3,Gender,Surge_Pricing_Type
0,T0005689460,6.77,1.0,1.0,2.42769,0.0,0,3.90500,0,40.0,46,60,1,1
1,T0005689461,29.47,1.0,10.0,2.78245,1.0,0,3.45000,0,38.0,56,78,0,1
2,T0005689464,41.58,NaN,10.0,NaN,NaN,4,3.50125,2,NaN,56,77,0,1
3,T0005689465,61.56,2.0,10.0,NaN,NaN,0,3.45375,0,NaN,52,74,0,2
4,T0005689467,54.95,2.0,10.0,3.03453,1.0,0,3.40250,4,51.0,49,102,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131657,T0005908509,11.72,3.0,1.0,2.74229,0.0,0,3.28500,0,61.0,47,76,0,2
131658,T0005908510,74.81,2.0,7.0,2.81059,2.0,0,0.44500,0,NaN,63,88,0,1
131659,T0005908512,40.17,2.0,10.0,2.99565,1.0,0,3.33625,0,NaN,48,75,1,1
131660,T0005908513,46.88,1.0,4.0,3.04744,0.0,1,4.15750,1,47.0,54,79,0,1


In [65]:
features = np.setdiff1d(train.columns, ['Trip_ID', 'Surge_Pricing_Type']) # returns set difference of the two arrays that have been 
                                                                          #passed as the arguments. Feature extraction, all columns except first and last       
params = {"objective": "multi:softmax","booster": "gbtree", "nthread": 4, "silent": 1,
                "eta": 0.08, "max_depth": 10, "subsample": 0.9, "colsample_bytree": 0.5,
                "min_child_weight": 1, "num_class": 3,
                 "tree_method": "exact"}

In [55]:
?xgb.train

In [91]:
dtrain = xgb.DMatrix(X_train[features], X_train['Surge_Pricing_Type'], missing=np.nan)
dtest = xgb.DMatrix(X_test[features], missing=np.nan)

C:\Users\Nishant\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\Nishant\Anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


In [92]:
nrounds = 200
watchlist = [(dtrain, 'train')]

bst = xgb.train(params, dtrain, num_boost_round=nrounds, evals=watchlist, verbose_eval=20)
test_preds = bst.predict(dtest)

submit = pd.DataFrame({'Trip_ID': test['Trip_ID'], 'Surge_Pricing_Type': test_preds+1})
submit.to_csv("XGB.csv", index=False)

[0]	train-merror:0.346592
[20]	train-merror:0.264776
[40]	train-merror:0.256103
[60]	train-merror:0.245325
[80]	train-merror:0.235231
[100]	train-merror:0.225016
[120]	train-merror:0.215977
[140]	train-merror:0.207159
[160]	train-merror:0.199275
[180]	train-merror:0.192622
[199]	train-merror:0.186584


In [ ]:
features = np.setdiff1d(train.columns, ['Trip_ID', 'Surge_Pricing_Type']) # returns set difference of the two arrays that have been 
                                                                          #passed as the arguments. Feature extraction, all columns except first and last       
params = {"objective": "multi:softmax","booster": "gbtree", "nthread": 4, "silent": 1,
                "eta": 0.08, "max_depth": 6, "subsample": 0.9, "colsample_bytree": 0.7,
                "min_child_weight": 1, "num_class": 3,
                "seed": 2016, "tree_method": "exact"}